# LLM 에 도구 바인딩(Binding Tools)

LLM 모델이 도구(tool) 를 호출할 수 있으려면 chat 요청을 할 때 모델에 도구 스키마(tool schema) 를 전달해야 합니다. 

도구 호출(tool calling) 기능을 지원하는 LangChain Chat Model 은 `.bind_tools()` 메서드를 구현하여 LangChain 도구 객체, Pydantic 클래스 또는 JSON 스키마 목록을 수신하고 공급자별 예상 형식으로 채팅 모델에 바인딩(binding) 합니다.

바인딩된 Chat Model 의 후속 호출은 모델 API에 대한 모든 호출에 도구 스키마를 포함합니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## LLM에 바인딩할 Tool 정의

실험을 위한 도구(tool) 를 정의합니다.

- `get_department` : 스터디 참석자들의 팀을 조회하는 함수 
- `get_word_length` : 단어의 길이를 반환하는 함수
- `add_function` : 두 숫자를 더하는 함수
- `naver_news_crawl` : 네이버 뉴스 기사를 크롤링하여 본문 내용을 반환하는 함수

**참고**
- 도구를 정의할 때 `@tool` 데코레이터를 사용하여 도구를 정의합니다.
- docstring 은 가급적 영어로 작성하는 것을 권장합니다.

In [3]:
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool


# 도구를 정의합니다.
@tool
def get_department(user: str) -> int:
    """사용자의 부서를 조회하는 데 사용된다."""
    
    # DB 혹은 API를 통해 사용자의 팀을 조회한다. 
    # 여기서는 map으로 관리
    department_map = {
        "이석준": "B팀",
        "나유경": "B팀",
        "정은영": "B팀",
        "서정현": "B팀",
        "송민정": "C팀",
        "김용진": "C팀",
        "김용익": "C팀",
        "조승열": "C팀",
        "노용주": "C팀",
        "장미현": "C팀",
        "김경태": "C팀",
        "이예지": "C팀",
        "김호진": "D팀",
        "권은홍": "E팀",
        "서태희": "E팀",
        "배기웅": "E팀",
        "이승엽": "T팀"
    }
    
    return department_map.get(user, "소속 팀을 찾을 수 없습니다.")


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b


@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"

@tool
def get_current_time():
    """현재 날짜 시간을 조회합니다."""
    return datetime.now()

tools = [get_department, get_word_length, add_function, naver_news_crawl, get_current_time]

## bind_tools() 로 LLM 에 도구 바인딩

llm 모델에 `bind_tools()` 를 사용하여 도구를 바인딩합니다.

In [4]:
from langchain_openai import ChatOpenAI

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

실행결과를 확인합니다. 

결과는 `tool_calls` 에 저장됩니다. 따라서, `.tool_calls` 를 확인하여 도구 호출 결과를 확인할 수 있습니다.

**참고**
- `name` 은 도구의 이름을 의미합니다.
- `args` 는 도구에 전달되는 인자를 의미합니다.

In [7]:
llm.invoke("What is the length of the word '이승엽'?")

AIMessage(content="The word '이승엽' consists of 3 syllables, which are 이, 승, and 엽. In terms of characters, it has 6 individual Hangul characters.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 20, 'total_tokens': 60, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'stop', 'logprobs': None}, id='run-100fedff-e9f3-4670-a0ec-76708c2c2078-0', usage_metadata={'input_tokens': 20, 'output_tokens': 40, 'total_tokens': 60, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
result = llm_with_tools.invoke("What is the length of the word '이승엽'?")
print(result)
print(result.model_dump_json())

content='' additional_kwargs={'tool_calls': [{'id': 'call_uxK009Rhx6W4tNATczd6xwwt', 'function': {'arguments': '{"word":"이승엽"}', 'name': 'get_word_length'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 156, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9654a743ed', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-f71f031e-f701-47fb-9bee-9795d1350d1f-0' tool_calls=[{'name': 'get_word_length', 'args': {'word': '이승엽'}, 'id': 'call_uxK009Rhx6W4tNATczd6xwwt', 'type': 'tool_call'}] usage_metadata={'input_tokens': 156, 'output_tokens': 19, 'total_tokens': 175, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
{"cont

In [10]:
# 실행 결과
result.tool_calls

[{'name': 'get_word_length',
  'args': {'word': '이승엽'},
  'id': 'call_uxK009Rhx6W4tNATczd6xwwt',
  'type': 'tool_call'}]

In [11]:
llm_with_tools.invoke("이승엽은 어느 팀에 있어?").tool_calls

[{'name': 'get_department',
  'args': {'user': '이승엽'},
  'id': 'call_D7R8TuMqNxq836xNfV8luIYu',
  'type': 'tool_call'}]

다음으로는 `llm_with_tools` 와 `JsonOutputToolsParser` 를 연결하여 `tool_calls` 를 parsing 하여 결과를 확인합니다.

In [12]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# 도구 바인딩 + 도구 파서
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

# 실행 결과
tool_call_results = chain.invoke("What is the length of the word '서정현'?")

In [13]:
print(tool_call_results)

[{'args': {'word': '서정현'}, 'type': 'get_word_length'}]


실행 결과는 다음과 같습니다.

**참고**
- `type`: 도구의 이름
- `args`: 도구에 전달되는 인자

In [14]:
print(tool_call_results, end="\n\n==========\n\n")
# 첫 번째 도구 호출 결과
single_result = tool_call_results[0]
# 도구 이름
print(single_result["type"])
# 도구 인자
print(single_result["args"])

[{'args': {'word': '서정현'}, 'type': 'get_word_length'}]


get_word_length
{'word': '서정현'}


도구 이름과 일치하는 도구를 찾아 실행합니다.

In [15]:
tool_call_results[0]["type"], tools[0].name

('get_word_length', 'get_department')

`execute_tool_calls` 함수는 도구를 찾아 args 를 전달하여 도구를 실행합니다.

즉, `type` 은 도구의 이름을 의미하고 `args` 는 도구에 전달되는 인자를 의미합니다.

In [16]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    # 도구 호출 결과 리스트를 순회합니다.
    for tool_call_result in tool_call_results:
        # 도구의 이름과 인자를 추출합니다.
        tool_name = tool_call_result["type"]  # 도구의 이름(함수명)
        tool_args = tool_call_result["args"]  # 도구에 전달되는 인자

        # 도구 이름과 일치하는 도구를 찾아 실행합니다.
        # next() 함수를 사용하여 일치하는 첫 번째 도구를 찾습니다.
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구를 찾았다면 해당 도구를 실행합니다.
            result = matching_tool.invoke(tool_args)
            # 실행 결과를 출력합니다.
            print(f"[실행도구] {tool_name} [Argument] {tool_args}\n[실행결과] {result}")
        else:
            # 일치하는 도구를 찾지 못했다면 경고 메시지를 출력합니다.
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")


# 도구 호출 실행
# 이전에 얻은 tool_call_results를 인자로 전달하여 함수를 실행합니다.
execute_tool_calls(tool_call_results)

[실행도구] get_word_length [Argument] {'word': '서정현'}
[실행결과] 3


## bind_tools + Parser + Execution

이번에는 일련의 과정을 한 번에 실행합니다.

- `llm_with_tools` : 도구를 바인딩한 모델
- `JsonOutputToolsParser` : 도구 호출 결과를 파싱하는 파서
- `execute_tool_calls` : 도구 호출 결과를 실행하는 함수

**흐름 정리**
1. 모델에 도구를 바인딩
2. 도구 호출 결과를 파싱
3. 도구 호출 결과를 실행

In [19]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# bind_tools + Parser + Execution
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

In [20]:
chain.invoke("서정현은 어느 팀에 있어?")

[실행도구] get_department [Argument] {'user': '서정현'}
[실행결과] B팀


In [22]:
# 실행 결과
chain.invoke("What is the length of the word '서정현'?")

[실행도구] get_word_length [Argument] {'word': '서정현'}
[실행결과] 3


In [23]:
# 실행 결과
chain.invoke("114.5 + 121.2")
print(114.5 + 121.2)

[실행도구] add_function [Argument] {'a': 114.5, 'b': 121.2}
[실행결과] 235.7
235.7


In [24]:
##########
#### 이건 될까?
chain.invoke("114.5 - 121.2")
print(114.5 - 121.2)

[실행도구] add_function [Argument] {'a': 114.5, 'b': -121.2}
[실행결과] -6.700000000000003
-6.700000000000003


In [25]:
# 실행 결과
chain.invoke(
    "뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
)

[실행도구] naver_news_crawl [Argument] {'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}
[실행결과] [미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국채지수(WGBI) 에 편입될 지 관심. 주식의 경우 지난 2009년

## bind_tools > Agent & AgentExecutor 로 대체

`bind_tools()` 는 모델에 사용할 수 있는 스키마(도구)를 제공합니다. 

`AgentExecutor` 는 실제로 llm 호출, 올바른 도구로 라우팅, 실행, 모델 재호출 등을 위한 실행 루프를 생성합니다.

**참고**
- `Agent` 와 `AgentExecutor` 에 대해서는 다음 장에서 자세히 다룹니다.

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [28]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 이전에 정의한 도구 사용
tools = [get_department, get_word_length, add_function, naver_news_crawl, get_current_time]

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [29]:
# Agent 실행
result = agent_executor.invoke({"input": "서정현은 어느팀에 있어?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_department` with `{'user': '서정현'}`


B팀서정현은 B팀에 있습니다.

> Finished chain.
서정현은 B팀에 있습니다.


In [30]:
# Agent 실행
result = agent_executor.invoke({"input": "How many letters in the word `서정현`?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': '서정현'}`


3The word `서정현` has 3 letters.

> Finished chain.
The word `서정현` has 3 letters.


In [31]:
result = agent_executor.invoke({"input": "지금 몇시야?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_current_time` with `{}`


2025-03-31 14:42:37.107036현재 시간은 2025년 3월 31일 14시 42분입니다.

> Finished chain.
현재 시간은 2025년 3월 31일 14시 42분입니다.


In [32]:
# Agent 실행
result = agent_executor.invoke({"input": "114.5 + 121.2 의 계산 결과는?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7114.5 + 121.2의 계산 결과는 235.7입니다.

> Finished chain.
114.5 + 121.2의 계산 결과는 235.7입니다.


한 번의 실행으로 끝나는 것이 아닌, 모델이 자신의 결과를 확인하고 다시 자신을 호출하는 과정을 거칩니다.

In [33]:
# Agent 실행
result = agent_executor.invoke(
    {"input": "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"}
)

# 결과 확인
print(result["output"])
print("==========\n")
print(114.5 + 121.2 + 34.2 + 110.1)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 235.7, 'b': 34.2}`


269.9
Invoking: `add_function` with `{'a': 269.9, 'b': 110.1}`


380.0114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

> Finished chain.
114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

380.0


이번에는 뉴스 결과를 크롤링 해서 요약 해달라는 요청을 수행합니다.


In [ ]:
result = agent_executor.invoke(
    {
        "input": "뉴스 기사를 요약해 줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
    }
)
print(result["output"])